In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [3]:
train = pd.read_csv('C:/Users/G/Documents/UoEssex/train.csv')
test = pd.read_csv('C:/Users/G/Documents/UoEssex/test.csv')

In [4]:
train_X = train.drop('SalePrice', axis=1)
train_y = train.SalePrice
test_X = test

In [5]:
onehot_train_X = pd.get_dummies(train_X)
onehot_test_X = pd.get_dummies(test_X)
train_X, test_X = onehot_train_X.align(onehot_test_X, join='left', axis=1)

In [6]:
my_imputer = SimpleImputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

In [7]:
reg = LinearRegression()
cv_scores = cross_val_score(reg, train_X, train_y, cv=5)
print(cv_scores)

[0.82273403 0.82015775 0.79383245 0.88785895 0.64306096]


In [8]:
reg = LinearRegression()
reg.fit(train_X, train_y)
predictions = reg.predict(test_X)


In [9]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice':predictions})
my_submission.to_csv('BASEsubmission.csv', index=False)